# Identifikace Jazyka

## Data

Data jsou zpracovány z webové stránky https://www.gutenberg.org/browse/languages/nl, kde jsou vybrány data ze tří jazyků - angličtiny,holandštiny a italštiny. 

Data pro angličtinu byly zpracovány z webovek:

https://www.gutenberg.org/cache/epub/75783/pg75783-images.html
https://www.gutenberg.org/cache/epub/75780/pg75780-images.html
https://www.gutenberg.org/cache/epub/8420/pg8420-images.html
https://www.gutenberg.org/cache/epub/75781/pg75781 images.html
https://www.gutenberg.org/cache/epub/1237/pg1237-images.html

Data pro holandštinu jsou zpracovány z:

https://www.gutenberg.org/cache/epub/61324/pg61324-images.html
https://www.gutenberg.org/cache/epub/58563/pg58563-images.html
https://www.gutenberg.org/cache/epub/48413/pg48413-images.html
https://www.gutenberg.org/cache/epub/65697/pg65697-images.html
https://www.gutenberg.org/cache/epub/63728/pg63728-images.html

Data pro italštinu jsou zpracovány z:

https://www.gutenberg.org/cache/epub/21425/pg21425-images.html
https://www.gutenberg.org/cache/epub/38637/pg38637-images.html
https://www.gutenberg.org/cache/epub/50674/pg50674-images.html
https://www.gutenberg.org/cache/epub/49626/pg49626-images.html
https://www.gutenberg.org/cache/epub/56890/pg56890-images.html

## Informace o datech

Data jsou postupně stažená z webové stránky a čištěna od nečistých dat, kde čistě data považujeme LATIN písmena. Počet získaných tokenů na jazyk jsou rozloženy rovnoměrně a počet tokenů získané pomocí knihovny Sacremoses je po řadě jazyků 322607,322607,322620 a velikost v bytech 2860824,2582328,2875224.

Data byly rozděleny v poměru 80 procent,10 procent, a zbytek po řade na trénovací,heldout a testovací data.

## ngramy

Pro každý jazyk jsme vypočítali pravděpodobnost unigramů,bigramů a trigramů pomocí vzorce P = c(h,w) / c(h), kde pro unigram P = c(w) / c(všech unigramů).

Pět nejčastějších trigramů angličtiny: the,and,her,hat,tha

Pět nejčastějších trigramů holandštiny: een,den,aar,het,zij

Pět nejčastějších trigramů italštiny: che,ell,lla,del,per

## Smoothing

Použiejem vzorec P(wIh) = ( c(w,h) + lambda) / (c(h) + lambda*V), kde lambdu zvolíme, tak aby trigramy v heldout datech měli co nejmenší cross entropii.

nejlepší lambda pro angličtinu,holandštinu, je zvolena: 0.001

## Testování

Cross entropie na trénovacích datech jsou

angličtina: 5.1561375174680695

holandština: 4.525547780503082

italština: 5.082175549172737


A máme vytvořenou funkci Classificator.predict(languages,input_filename) na libovolně zvolený text, kde je nutné vyplnit text unknown.txt zvoleným textem. Testy na testových datech vyšli správně a jazyky byly správně přiřazeny. testový text anglický k angličtině, testový test hokandský k holandštině a testový text italský k italštině

## Submission File

your name: Jáchym Mraček
Chosen languages: en,nl,it
sripts: Všechny jazyky jsou v latince, kde italština a holandština mohou obshaovat přehlásky.
data sources: Stejné jako v úvodu.
data size in bytes: 2860824,2582328,2875224
data size in tokens: 322607,322607,322620
Percentage of heldout OOV tokens: 29,22,28
Percentage of heldout OOV tokens: 29,22,28
5 most frequent trigrams:

Pět nejčastějších trigramů angličtiny: the,and,her,hat,tha

Pět nejčastějších trigramů holandštiny: een,den,aar,het,zij

Pět nejčastějších trigramů italštiny: che,ell,lla,del,per

Best smoothing parameter values: 0.01,0.01,0.01

Cross-entropy before smoothing: 3.565211720588061,3.753433935283187,3.1655068522328453

Cross-entropy after smoothing: 5.1561375174680695, 4.525547780503082,5.082175549172737

Random Sentence: ciao come stai, sto benissimo, come stai?
Code output on random sentence: ('it', -52.45840302143219)

OMLOUVÁM SE ://///, NĚKTERÉ INFORMACE JSEM NEUVEDL VE FORMULÁŘI SPRÁVNĚ. V KODU UVÁDÍM, ŽE JSEM NĚJAKÉ VÝPOČTY DOPLNIL PO DEADLINE, PROMIŇTE MI TO OMLOUVÁM SE. PROSÍM NEBERTE MI TOTO ŘEŠENÍ JAKO PO DEADLINE. ŘEŠENÍ PŘED DEADLINE JE: https://github.com/JachymMracek/NPFL_HW1. NAPIŠTE PROSÍM V HODNOCENÍ, JAK JSTE HODNOTILI DOPLŃUJÍCÍ INFORMACE. DĚKUJI

In [ ]:
import unicodedata
from sacremoses import MosesTokenizer
from collections import Counter
import math
import sys

class Chars:
    def __init__(self):
        self.symbols = set()
    
    def is_symbol_in(self,char):
        if char in self.symbols:
            return True
    
    def is_correct(self,char) -> bool:
        if char.isalpha() and unicodedata.name(char, "").startswith("LATIN"):
            self.symbols.add(char)
            return True
    
        else:
            return False

class Formator:
    def correct_text(filename,tag,not_print,SPACE = " "):
        chars = Chars()

        text = ""

        with open(filename,"r" ,encoding='utf-8') as language_file:
                for line in language_file:
                    for char in line:
                        
                        if chars.is_symbol_in(char) or chars.is_correct(char):
                            text += char
                        
                        elif char == SPACE:
                            text += SPACE

        tokens = Formator.__tokenize(text,tag,not_print)

        return tokens
    
    def __tokenize(text, tag,not_print,COUNT_WORDS=322607):
        mt = MosesTokenizer(lang=tag)
        tokens = mt.tokenize(text)
        
        word_counts = Counter(tokens)
        
        most_common_words = word_counts.most_common()
        
        tokens_available = []
        counted_words = 0
        position = 0
        
        while counted_words < COUNT_WORDS:
            if position >= len(most_common_words):
                break

            word, count = most_common_words[position]
            
            tokens_available.extend([word] * count)
            counted_words += count
            position += 1
        
        bytes = sys.getsizeof(tokens_available)

        if not_print:
            print("count bytes: " +  str(bytes))
            print("count of words: " +  str(counted_words))
        
        return tokens_available
    
    def get_index(quotient,count_words):
        return int((quotient*count_words) // 1)
    
    def split(tokens,TRAIN_QUOTIENT = 0.8,HELDOUT_QUOTIENT = 0.1):
        count_words = len(tokens)

        train_index = Formator.get_index(TRAIN_QUOTIENT,count_words)
        heldout_index = train_index + Formator.get_index(HELDOUT_QUOTIENT,count_words)

        train = tokens[:train_index + 1]
        heldout = tokens[train_index + 1:heldout_index+1]
        test = tokens[heldout_index+1:]

        return train,heldout,test
    
    def get_ngrams(tokens, n):
        ngrams = []

        for token in tokens:
            token_count = len(token)

            for i in range(token_count - n + 1):
                ngrams.append(token[i:i+n])
                
        return ngrams

class Model:
    N = 3
    def __init__(self):
        self.V = None
        self.best_lambda = None
        self.counter = Counter()
    
    def __report(self,trigrams_count,BEST=5):
        trigrams = [(trigram, count) for trigram, count in self.counter.items() if len(trigram) == 3]

        best_trigrams = sorted(trigrams, key=lambda x: x[1], reverse=True)[:BEST]

        for ranking, (trigram, count) in enumerate(best_trigrams):
            print(str(ranking + 1)  + "th" + " most common trigram is " + trigram + 
                  " with count " + str(count) + " and relative frequence " + str(round(count/trigrams_count,2)))
    
    def fit(self,train_tokens,tokens_heldout,tokens_test):
        ngram_probability = {}
        trigrams_count = None

        for n in range(1,self.N+1):
            ngrams = Formator.get_ngrams(train_tokens,n)
            n_counter = Counter(ngrams)
            self.counter += n_counter

            self.__probability_ngram(ngrams,ngram_probability,n)

            if n == self.N:
                trigrams_count = sum(n_counter.values())
        
        self.__report(trigrams_count)

        trigrams_heldout = Formator.get_ngrams(tokens_heldout,self.N)
        trigrams_test = Formator.get_ngrams(tokens_test,self.N)

        self.best_lambda = self.__get_best_lambda(trigrams_heldout)

        self.cross_entropy_trigram_not_smoothing(ngram_probability,trigrams_test)

    def cross_entropy_trigram_not_smoothing(self,ngram_probability,trigrams_test): # po deadline
        trigrams = {trigram: count for trigram, count in ngram_probability.items() if len(trigram) == 3 and trigram in trigrams_test }
        cross_entropy = 0

        for _,p in trigrams.items():
            cross_entropy += math.log2(p)

        cross_entropy *= -1 / len(trigrams)

        print("initial cross entropy test: " + str(cross_entropy))
    
    def __probability_ngram(self,ngrams,ngram_probability,n): 
        for ngram in ngrams:
            if n == 1:
                history = len(self.counter)
                self.V = history
            else:
                history = self.counter[ngram[:-1]]
        
            ngram_probability[ngram] = self.counter[ngram] / history

    def __smoothing_probability(self, ngram, lambda_choosen):

        if ngram not in self.counter:
            count_ngram = 0
        else:
            count_ngram = self.counter[ngram]

        if ngram[:-1] not in self.counter:
            count_history = 0
        else:
            count_history = self.counter[ngram[:-1]]

        return (count_ngram + lambda_choosen) / (count_history + lambda_choosen * self.V)

    def probability_language(self, ngrams_test):
        log_p_language = 0  # GPT Markov chain mi poddíkal na nulu.

        for ngram in ngrams_test:
            prob = self.__smoothing_probability(ngram, self.best_lambda)
            log_p_language += math.log(prob)
            
        return log_p_language
    
    def cross_entropy(self,ngrams,lambda_choosen):
        cross_entropy = 0

        for ngram in ngrams:
            p = self.__smoothing_probability(ngram,lambda_choosen)

            if p < 0 or p > 1:
                cross_entropy = float("inf")
            else:
                cross_entropy += math.log2(p)

        cross_entropy *= -1 / len(ngrams)

        return cross_entropy
    
    def __get_best_lambda(self,ngrams):
        best_lambda = None
        best_cross_entropy = float("inf")

        lambdas = [x * 0.001 for x in range(1, 1000)]

        for lambda_choosen in lambdas:
            cross_entropy = self.cross_entropy(ngrams,lambda_choosen)
        
            if  best_cross_entropy > cross_entropy:
                best_cross_entropy = cross_entropy
                best_lambda = lambda_choosen

        return best_lambda

class Language:
    def get_oov(self,train,trigrams): #PO deadline
        oov = 0
        was = 0
        for trigram in trigrams:
            was += 1
            if trigram not in set(train):
                oov += 1

        print(oov / len(trigrams))

    def __init__(self,tag,filename):
        self.model = Model()
        self.tag = tag

        train_tokens,heldout_tokens,self.test_tokens = self.__read_data(tag,filename)

        train_tokens_trigram = Formator.get_ngrams(train_tokens,self.model.N)
        heldout_tokens_trigram = Formator.get_ngrams(self.test_tokens,self.model.N)
        test_tokens_trigram = Formator.get_ngrams(self.test_tokens,self.model.N)

        # self.get_oov(train_tokens_trigram,heldout_tokens_trigram) zakomentoval jsme to protože to trvá moc dlouho
        # self.get_oov(train_tokens_trigram,test_tokens_trigram) vypočty jsou po deadline

        self.model.fit(train_tokens,heldout_tokens,self.test_tokens)

        print("Test cross entropy of language " + tag + " is " + str(self.model.cross_entropy(Formator.get_ngrams(self.test_tokens,self.model.N),self.model.best_lambda)))
    
    def __read_data(self,tag,datafilename):
        print("Language informations: ")
        tokens = Formator.correct_text(datafilename,tag,False)
        return Formator.split(tokens)

class Classificator:
    def predict(languages:list[Language],filename):
        predictions = []

        for language in languages:
            tokens = Formator.correct_text(filename,language.tag,True)

            ngrams = Formator.get_ngrams(tokens, Model.N)

            p = language.model.probability_language(ngrams)

            predictions.append((language.tag,p))
        
        predictions_sorted = sorted(predictions, key=lambda x: x[1], reverse=True)
        print(predictions_sorted[0])

        return predictions

if __name__ == "__main__":
    input_filename = "unknown.txt"
    filenames = ["english.txt","dutch.txt","italian.txt"]
    tags = ["en","nl","it"]
    languages = []

    for i in range(3):
        languages.append(Language(tags[i],filenames[i]))
    
    # for i in range(3):
       # Classificator.predict(languages,languages[i].test_tokens)
    
    Classificator.predict(languages,input_filename)